In [1]:
# initial setup
import os
import sys
import json
from collections import defaultdict
os.environ['DATABASE_URL'] = 'postgres:///awrdata'

%matplotlib inline
import matplotlib.pyplot as plt
os.chdir('/users/dane/src/datatools/ipy/notebooks/pt/dev/')
sys.path.append('/users/dane/src/datatools/analytics/')

import pertest.util
from pertest.util import *

import numpy as np
import pandas as pd

# data is linked to ~/data
df = pd.read_csv('../data/v14_b60-73.csv')
df['value'] = df['t_value']  # required for level_shift
print(len(df), 'records read')
# keep only time based metric data
odf = df[df.m_name.apply(lambda x: ('$T' not in x) and ('$M' not in x))]
print(len(odf), 'records')
print(len(odf.m_id.unique()), 'unique metrics')

1661364 records read
618447 records
5572 unique metrics


# Non-Time/Memory Metrics

In [2]:
# build metric dataframe
tdf = odf.sort_values(by='build')  # make sure it's in the right order
d = []
for m_id, items in tdf.groupby('m_id'):
    rec = items.iloc[0]
    f15 = items[:15]
    l15 = items[-15:]
    f15m = f15.t_value.median()
    m = items.t_value.median()
    # do all points fall within .1 of mean
    avg = items.t_value.mean()
    mx = items.t_value.max()
    mn = items.t_value.min()
    minfo = dict(
        m_id=rec.m_id,
        name=rec.m_name,
        test_case=rec.test_case,
        f15median=f15m,
        f15mean=f15.t_value.mean(),
        f15range=f15.t_value.max() - f15.t_value.min(),
        l15median=l15.t_value.median(),
        l15mean=l15.t_value.mean(),
        median=m,
        mean=avg,
        max=mx,
        min=mn,
        range=mx - mn,
    )
    d.append(minfo)
metrics = pd.DataFrame(d)
pertest.util.df = odf
pertest.util.metrics = metrics
metrics.head()

,f15mean,f15median,f15range,l15mean,l15median,m_id,max,mean,median,min,name,range,test_case
0,1.0,1.0,0.0,1.0,1.0,72873,1.0,1.0,1.0,1.0,AxPerf:Freqs_Sim,0.0,Microstrip_to_Stripline@Microstrip_to_Striplin...
1,67.0,67.0,0.0,67.0,67.0,72883,67.0,67.0,67.0,67.0,AxPerf:Iterations,0.0,Microstrip_to_Stripline@Microstrip_to_Striplin...
2,0.0,0.0,0.0,0.0,0.0,72889,0.0,0.0,0.0,0.0,AxPerf:Sim Mode,0.0,Microstrip_to_Stripline@Microstrip_to_Striplin...
3,8.0,8.0,0.0,8.0,8.0,72893,8.0,8.0,8.0,8.0,AxPerf:Res_Threads,0.0,Microstrip_to_Stripline@Microstrip_to_Striplin...
4,6.0,6.0,0.0,6.0,6.0,72898,6.0,6.0,6.0,6.0,AxPerf:Freqs_Sim,0.0,nRT8_Stripline_DeEmbedding_Single_Trace_Finite...


## Other Metrics

### Fixed value metrics

Several other parameters such as Model, Ncp, Np, Order and Status don't have specific values but should not change and will be MEAN_EX.

In [6]:
metric_settings = {}
for m_id, items in odf.groupby('m_id'):
    row = items.iloc[0]
    (file, name) = row.m_name.split(':')
    if ('$H' in name) or (name in ['Model', 'Ncp', 'Np', 'Order', 'Status', 'Iterations']):
        metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                                     meas_type='MEAN_EX', mean=row.t_value)
len(metric_settings)

2584

### Error Metrics

Some metrics are errors.  In this case, we have an upper bound of -30 and an error tolerance of +/- 15.  Some of these metrics are > 0 which should be illegal so those will be made pass metrics.

In [7]:
c = Counter()
x = []
for m_id, items in odf.groupby('m_id'):
    row = items.iloc[0]
    (file, name) = row.m_name.split(':')
    if name in ['Max_err', 'RMS_err', 'Rfun0', 'Rfun1', 'Rfun2', 'Rfun3']:
        c['Num'] += 1
        v1 = row.t_value
        if v1 == 0:
            failed = items[items.t_value !=0]
            if len(failed) > 0:
                c['Failed 0'] += 1
            metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                             meas_type='VAL_EX', value=0)
        elif v1 > 0:
            # need to know what should really happen to these
            metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                         meas_type='PASS')
        else:
            f15mean = items[:15].t_value.mean()
            upper = min(-30, f15mean+10)
            lower = f15mean - 10
            failed = items[(items.t_value > upper) | (items.t_value < lower)]
            if len(failed) > 0:
                x.append(m_id)
                c['Failed window'] += 1
            metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                             meas_type='MEAN_ABS', mean=f15mean, upper=(upper-f15mean), lower=(f15mean-lower))
print(c)

Counter({'Num': 1020, 'Failed window': 95})


In [8]:
len(metric_settings)

3604

For the phase errors:

* If the value is 0, AFS did not run so the value should remain 0
* Any value > 0.2 is a failure or any change greater than 0.05

In [9]:
c = Counter()
x = []
for m_id, items in odf.groupby('m_id'):
    row = items.iloc[0]
    (file, name) = row.m_name.split(':')
    if name == 'Ph_err':
        c['Num'] += 1
        v1 = row.t_value
        if v1 == 0:
            failed = items[items.t_value !=0]
            if len(failed) > 0:
                c['Failed 0'] += 1
            metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                         meas_type='VAL_EX', value=0)
        else:
            f15mean = items[:15].t_value.mean()
            upper = min(.2, f15mean+.05)
            lower = f15mean - .05
            failed = items[(items.t_value > upper) | (items.t_value < lower)]
            metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case, current_sequence=0,
                 meas_type='MEAN_ABS', mean=f15mean, upper=(upper-f15mean), lower=(f15mean-lower))
            if len(failed) > 0:
                x.append(m_id)
                c['Failed window'] += 1
print(c)

Counter({'Num': 170, 'Failed window': 1})


In [10]:
len(metric_settings)

3774

## Unused metrics

the V_c and V_i metrics are not used and will be set to PASS.

In [11]:
for m_id, items in odf.groupby('m_id'):
    row = items.iloc[0]
    (file, name) = row.m_name.split(':')
    if name.startswith('V_'):
        metric_settings[m_id] = dict(id=int(m_id), name=row['m_name'], test_case=row.test_case,
                                     current_sequence=0, meas_type='PASS')
len(metric_settings)

3980

# Conclusion

Based on the rules supplied by Pekka we will set the metrics as above.

In [12]:
data = list(metric_settings.values())
print(type(data))
print(len(data))
with open('other_settings.json', 'w') as fp:
    fp.write(json.dumps(data, indent=2))

    

<class 'list'>
3980
